# Build a super small Blockchain in Python in 100 lines

### A step by step tutorial

We have learned a lot of theory about Blockchain and cryptocurrency in class. To better understand how Blockchain works, I am going to build a very small Blcokchain in Python and explain them in detail here. Unlike other tutorials, the Blockchain I am going to build is very simple and omit some features of Blockchain. But the advantage is that the program will be extremely short and people with zero coding foundation can understand it.

In first three steps, we build a blockchain in one computer. We may need some basic HTTP and back-end knowledge to fully understand the code after step3

Requirement:  
python 3.6  
flask 1.0.2

## Step1: Define our block

As we only want a very basic blockchain, we only store five elements in our block. 
1. timestamp
2. index
3. data
4. previous hash
5. block hash

As we have learned in class, each block should have a hash which is a cryptographic hash of the block’s index, timestamp, data, and the hash of the previous block’s hash. The data can be anything we want to store. The hash function I am using is sha256 which has been introduced in the class

In [1]:
#import python library we will use
from datetime import datetime
import hashlib as hasher
import json

In [2]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
    
    #This function is used to print the block's index
    def showBlock(self):
        return json.dumps({
            'index': self.index,
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.hash
        })
    
    def hash_block(self):
        sha = hasher.sha256()
        seq = [str(self.index), str(self.timestamp), str(self.data), str(self.previous_hash)]
        sha.update(''.join(seq).encode('utf-8'))
        return sha.hexdigest()

**Code Explaination**  
(index, timestamp, data, previous_hash) are provded when we create a new block. The hash_block function is used to create the hash for this block. We firstly cast index, timestamp, data, previous_hash to be string and save them in ```seq```. Then we use sha256 function from Python library to hash all of (index, timestamp, data, previous_hash) into one single hash and the hash will be the hash of this block.

The function showBlock is used to show the information of the current block and will be used in the later part

## Step2: Create the blockchain

### Create genesis block

Each block needs hash from previous block, but first block (genesis block) does not have previous block. So we need to treat it specially and create it manually

In [3]:
def make_genesis_block():
    """Make the first block in a block-chain."""
    block = Block(index=0,
                  timestamp=datetime.now(),
                  data="Genesis Block",
                  previous_hash="0")
    return block

**Code Explaination**  
We create a new block using ```Block``` which we define in Step1. As we don't have hash from previous block, we simply give a random string e.g "0". ```datetime.now()``` gives the current time and as this is our first block, we set index to 0

### Link other blocks

Next, we need to write a function to build a new block based on the previous block. 

In [4]:
def next_block(pre_block, data=''):
    """Return next block in a block chain."""
    idx = pre_block.index + 1
    block = Block(index=idx,
                  timestamp=datetime.now(),
                  data='This is block {}'.format(idx),
                  previous_hash=pre_block.hash)
    return block

**Code Explaination**  
This function takes previous block as parameter. Firstly, the function gets previous block's index and increase it by 1 as the current block's index. As I have mentioned before, we need to embed previous block's hash to increase the integrity of the blockchain. This chain of hashes acts as cryptographic proof and helps ensure that once a block is added to the blockchain it cannot be replaced or removed.


### Test our code

We have finished a very basic version our the blockchain. Let's test it by creating 15 blocks and linking them together.

In [5]:
def run_simple_chain():
    """Test creating chain of 15 blocks."""
    blockchain = [make_genesis_block()]
    prev_block = blockchain[0]
    for _ in range(0, 15):
        block = next_block(prev_block, data='Change to anything you want')
        blockchain.append(block)
        prev_block = block
        print('Block {} added to blockchain at {}'.format(block.index, block.timestamp))
        print('Previous block\'s hash: {}'.format(block.previous_hash))
        print('Hash: {}\n'.format(block.hash))

**Code Explaination**  
We use list(a data structure in Python, similar with array) to store the blocks. We create 15 blocks using loop. Firstly, we create the genesis block. Then everytime after we create a new block using the previous block, we need to set the ```prev_block``` to the new block we create so that in the next iteration, we can use it to create new block. If we do not do this, all the blocks we build will be based on the genesis block.


In [6]:
run_simple_chain()

Block 1 added to blockchain at 2018-12-16 22:40:33.216532
Previous block's hash: 283d3c5a61395a9cc3cbfac7271e3d518bb008a1631b5d3f2cd0606982fb2f68
Hash: f93b5b2ce3e484f408ed68d69fe026e20340eff1d78c720da3c36e6b46f6f857

Block 2 added to blockchain at 2018-12-16 22:40:33.217167
Previous block's hash: f93b5b2ce3e484f408ed68d69fe026e20340eff1d78c720da3c36e6b46f6f857
Hash: 307c1169483ffe8f0d3cbe3bd660457217264c23bc66d834c5f1d30d8a9de87a

Block 3 added to blockchain at 2018-12-16 22:40:33.217283
Previous block's hash: 307c1169483ffe8f0d3cbe3bd660457217264c23bc66d834c5f1d30d8a9de87a
Hash: 2149f41c672801d563223e97b3160726aaf8c10d0310cb2ae9ed2b02ebf2e9de

Block 4 added to blockchain at 2018-12-16 22:40:33.217379
Previous block's hash: 2149f41c672801d563223e97b3160726aaf8c10d0310cb2ae9ed2b02ebf2e9de
Hash: 24d050f89a9c671647377656580faf3b027827ed1b5590b3e9d385b67de718a0

Block 5 added to blockchain at 2018-12-16 22:40:33.217464
Previous block's hash: 24d050f89a9c671647377656580faf3b027827ed1b5590b

As we can see, we add 15 blocks and each block store previous block's hash and use it to generate the new block

**----------------------------You may need some basic HTTP and back-end knowledge to fully understand the following parts----------------------------**

## Step3: Change data field to a transaction

In the previous simple version, the data field in each block is not used we set it to any random string. But noew we are going to change data field to a transaction. Same as Bitcoin, we want to use the transaction to keep tract of the sender, the receiver and amount. So the exmple of a transaction is  
```
{
  "from": "71238uqirbfh894-random-public-key-a-alkjdflakjfewn204ij",
  "to": "93j4ivnqiopvh43-random-public-key-b-qjrgvnoeirbnferinfo",
  "amount": 3
}
```

## Step4: Accept transactions from users

As we have been taught in the lecture, the blockchain system should be able to accept transactions from users and after verify the transaction, the transaction can be added to the blockchain system.

In step4, we are going add a funtion to accept transactions from users

Before we write the function, we need to import **flask** which is a Python backend framework. We will use flask help create the server to accept request from users

In [17]:
from flask import Flask
from flask import request
import requests
from sys import argv
node = Flask(__name__)

In [6]:
@node.route('/transaction', methods=['POST'])
def transaction():
    if request.method == "POST":
        #extract transaction data from POST request
        transaction = request.get_json()
        #store it in a list
        transactions.append(transaction)

        print('New Transaction')
        print(json.dumps(transaction))
        print(transactions)
        return json.dumps(list(transactions))

**Code Explaination**  
When the user send a POST request to the url "/transaction", this function will extract data from this request and store in a list called transaction

## Step5: Proof-of-Work and Mining

Now, we can keep track of the transactions data sent by users. The blockchain system should allow people to mine new blocks and earn new coins. In this step, we implement two functions, proof_of_work and mining.

### Proof-of-Work (PoW)

As we have learned in the class, the blockchain system does not want it to be too easy to mine the block, so it need some kind of difficulty of mining new blocks. We need a proof-of-work algorithm. 

A proof of work should be costly and time-consuming to produce but easy for others to verify and which satisfies certain requirements. Producing a proof of work can be a random process with low probability so that a lot of trial and error is required on average before a valid proof of work is generated. 

In [3]:
def proof_of_work(previous_proof):
    incrementor = previous_proof + 1
    while not (incrementor % 11 == 0 and incrementor % last_proof == 0):
        incrementor += 1
        
    return incrementor

**Code explaination**  
To simplifiy the system, we create a very simple PoW. The function will take in the proof number from last block and keep increasing the number until it's divisible by 11(you can change to any number you want) and the proof number of previous block. Once the miner finds this number, a new coin should be rewarded to the miner

### Mining

In [7]:
@node.route('/mine', methods=['GET'])
def mining():
    last_block = blockchain[-1]
    last_proof = last_block.data['proof-of-work']
    proof = proof_of_work(last_proof)
    transactions.append({"from": "network", "to": miner_address, "amount": 1})   
    mined_block = Block(
        index=last_block.index + 1,
        timestamp=datetime.now().isoformat(),
        data={
            'proof-of-work': proof,
            'transactions': list(transactions)
        },
        previous_hash=last_block.hash
    )

    blockchain.append(mined_block)
    transactions[:] = []
    return json.dumps([i.showBlock() for i in blockchain])

**Code Explaination**  
We simplify the mining process and this function will do the following steps:

1. get the proof of work from the last block in the blockchain
2. use function proof_of_work to find the a new proof number
3. add a new transaction (the reward given by the blockchain to the miner) to the transactions list
4. create a new block, which is used to store all current transactions accected by the system
5. append this block to the blockchain list
6. clear the transactions list, as we have keep all the transactions information to a new block in step5
7. return the blockchain list back to the miner


As we can see from the mining function, the data field of the block to a new form which is a JSON format data including two fields, proof-of-work and transactions. In order to be consistent, we need to change the make_genesis_block function. 

In [20]:
def make_genesis_block():
    """Make the first block in a block-chain."""
    block = Block(index=0,
                  timestamp=datetime.now(),
                  data= {
                    'proof-of-work': 0,
                    'transactions': []
                  },
                  previous_hash="0")
    return block

## Last Step: Consensus

Now we allows user to do two things  
1. they can send coins to each other by sending transactions to our server
2. the system can issue new coins to people when they finish a mining process. 

But we are doing this on one computer. Blockchain is decentralized, we need to make sure everyone in the network has the same chain. To do so, we can ask each node broadcast its chain to others and allow each node to verify other's chain so that the whole network comes to a *consensus* of what the current chain looks like. This is the **consensus algorithm**

To simplify our system, we define the consensus algorithm as follow:  
if a node's chain is diffrent from others, then the longest chain in the network will be the correct one and other shorter chains will be abandoned. If chains of all nodes are the same then, we continue

In [10]:
@node.route('/add_node', methods=['POST'])
def add_node():
    n = request.get_json()
    peer_nodes.add(n["node"])
    return json.dumps(list(peer_nodes))

**Code Explaination**  

When a new computer is added to the network, it should let other nodes in the network know new node joins the network. So the new computer should send a POST request to usrl "/add_node" with url of the new computer so that each node in the network can use this url to communicate with the new computer. All the urls of other nodes are stored in a list called peer_nodes

In [11]:
@node.route('/chains', methods=['GET'])
def get_chain():
    chain_to_send = blockchain
    ret = []
    for block in chain_to_send:
        new_block = {
            'index': block.index,
            'timestamp': block.timestamp,
            'data': block.data,
            'hash': block.hash}
        ret.append(new_block)
    return json.dumps(ret)

**Code Explaination**  

As we have explained above, when we do consensus, every node should get other node's chain and compare their length. When other nodes ask this computer what chain do you have by sending a GET request to "/chains", this function will get its current chain and send it back by transforming the chain list to a JSON object. The for loop in this function is used to convert the Block object into a dictionary object, as we can easily convert a list of dictionary to a JSON object.

More information about dictionary and JSON
https://www.w3schools.com/python/python_dictionaries.asp
https://www.w3schools.com/python/python_json.asp

In [12]:
def find_new_chains():
    other_chains = []
    for node_url in peer_nodes:
        print(node_url)
        block = requests.get(node_url + "/chains").content
        block = json.loads(block)
        other_chains.append(block)
        print(block)        
    return other_chains

**Code Explaination**  

As we have mentioned above, the peer_nodes is a set used to store all the urls of other nodes in the network. Also, if node1 want to know what chain does node2 has, it can send a GET request to 'url_node1/chains' and the get_chain function of node2 will send the chain that node2 has back to node1.

Now before we implement our consensus function, the last step is to ask every node in the network to send their chains to us so that we can compare which chain is the longest. The find_new_chains function basically tries to send request to other nodes. If we get chains back, we store them in a list called other_chains which we will use in our consensus function

In [14]:
def consensus():
    global blockchain
    other_chains = find_new_chains()
    longest_chain = blockchain
    flag = False
    for chain in other_chains:
        if len(longest_chain) < len(chain):
            flag = True
            longest_chain = chain
    if flag:
        tmp = []
        for i in longest_chain:
            tmp.append(Block(i['index'], i['timestamp'], i['data'], i['hash']))
        blockchain = tmp
        return
    blockchain = longest_chain

**Code Explaination**  
Finally, this is our short and simple consensus function. This function does two things:  
1. get other nodes' chain by calling find_new_chains() we implemented above
2. we compare the length of each chain and choose the longest chain as our chain.

The reason we want to use a flag is the find_new_chains() return a list of JSON format data. But our blockchain list consist of Block object. If there is longer chain in other nodes, we should convert the JSON data back to Block object

**Last final step**  
It seems we miss one final step,  where should we call the consensus function? 

When the user wants to know what chain a node has, the node should run the consensus() function and send chain back by calling get_chain() we implement above.


In [15]:
@node.route('/cur_chain', methods=['GET'])
def cur_chain():
    consensus()
    return get_chain()

## Last Last step: Initialization

In [25]:
miner_address = 'miner address ' + str(argv[1])
blockchain = [make_genesis_block()]
previous_block = blockchain[0]
transactions = []
peer_nodes = set()

- miner_address is used as the address to receive coin 
- reward  blockchain is used to store all the block this node currently has  
- previous block is the last block in the blockchain list
- transaction is a list used to store all transactions that users sent to this node. This list will be empty everytime this node mine a block
- peer_nodes is a set() used to store the urls of other nodes. The difference between set and list is that set is unique and will not have duplicate elements.


## Let's test our blockchain

#### 1. Launch two nodes using localhost with port 16001 and 16002. 

We now we call two notes 16001 and 16002. 
Open two terminal windows, and run one command in one window  
```bash
python aps1050.py 16002 
python aps1050.py 16001
```

In each windows, you should see the following information

```
 * Serving Flask app "aps1050" (lazy loading)
 * Environment: production
   WARNING: Do not use the development server in a production environment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:16001/ (Press CTRL+C to quit)
```

#### 2. Send request to "/add_node" so that 16001 and 16002 know who are in the network

Open another terminal windows and run these two commands one by one  
```bash
curl "localhost:16001/add_node" -X POST -H "Content-Type: application/json" -d '{"node": "http://127.0.0.1:16002"}'
curl "localhost:16002/add_node" -X POST -H "Content-Type: application/json" -d '{"node": "http://127.0.0.1:16001"}'
```

We should see a list of url when we run each command  
```bash
["http://127.0.0.1:16002"]
```

#### 3. Send two transactions to 16001

In the same windows, run these two commands one by one
```bash
curl "localhost:16001/transaction" -H "Content-Type: application/json" -d '{"from": "a", "to":"b", "amount": 3}' 
curl "localhost:16001/transaction" -H "Content-Type: application/json" -d '{"from": "b", "to":"c", "amount": 3}'
```
We should see a list of transaction of node16001 when we run each command
```bash
[{"from": "a", "to": "b", "amount": 3}][{"from": "a", "to": "b", "amount": 3}, {"from": "b", "to": "c", "amount": 3}]
```


#### 4. Mine a block using node16001
In order to put a block in the blockchain, we ask 16001 to mine a block

```bash
curl "localhost:16001/mine"
```

Now node16001 has 2 blocks, 
1. one genesis block(index 0)
2. Another block (index 1) has 3 transactions:  
   Two transactions we sent and one more contains the mining reward transaction

```bash
[{"index": 0, "timestamp": "2018-12-16T17:49:52.035854", "data": {"proof-of-work": 9, "transactions": []}, "previous_hash": "0", "hash": "595564d37e277e6066c534831f3d04d58c0dc86dc6fbb845417370882e1a7be0"}, {"index": 1, "timestamp": "2018-12-16T17:52:59.138048", "data": {"proof-of-work": 99, "transactions": [{"from": "a", "to": "b", "amount": 3}, {"from": "b", "to": "c", "amount": 3}, {"from": "network", "to": "miner address 16001", "amount": 1}]}, "previous_hash": "595564d37e277e6066c534831f3d04d58c0dc86dc6fbb845417370882e1a7be0", "hash": "b71a049ff5756b9617ac8810a1821c8dbdba8bede70c17d0a1d3e84ccf4f4976"}]
```


#### 5. What chain should node16002 have
We can find out what chain does node16002 have by runing this command in the same window

```bash
curl "localhost:16002/cur_chain"
```

Node16002 has only one genesis block, but when we send request to 16002 ask what chain does it have, it should ask all notes for their chains (we only have 16001 so far) and find that 16001 has the longest chain (comparing with all nodes including 16002), so 16002 will use 16001's chain. So node16002 has the same chain as node16001.

```bash
[{"index": 0, "timestamp": "2018-12-16T17:49:52.035854", "data": {"proof-of-work": 9, "transactions": []}, "hash": "adc07f0962b0f7713e405b90d057258db9fe1626723a5bc66669c9df0f9c3076"}, {"index": 1, "timestamp": "2018-12-16T17:52:59.138048", "data": {"proof-of-work": 99, "transactions": [{"from": "a", "to": "b", "amount": 3}, {"from": "b", "to": "c", "amount": 3}, {"from": "network", "to": "miner address 16001", "amount": 1}]}, "hash": "7b11b5b5784de063d7a1eba914dc5782b2f3908f8e41638208e60907f8364ddb"}, {"index": 2, "timestamp": "2018-12-16T17:59:19.097519", "data": {"proof-of-work": 198, "transactions": [{"from": "a", "to": "b", "amount": 3}, {"from": "b", "to": "c", "amount": 3}, {"from": "network", "to": "miner address 16001", "amount": 1}]}, "hash": "6ab5ca628b9974b98f40276687898978961992eb1eb89dcab75911d09876a30e"}]
```


## What can we improve?

What can the system currently do?
1. A node can receive transactions
2. A node can mine blocks and receive reward from the system
3. A node can ask others what chains do other nodes have
4. A node will replace the longest chain in the network with its original chain

What we are missing?  
1. Each user does not has his/her own wallet, so we can implement the wallet so each user can send, receive and store their coins
2. A user interface that normal users can iteract with the system. We are using command line to interact with nodes now which is hard to use.
3. A better PoW and consensus algorithm. We are using a very easy PoW and consensus algorithm which can be optimized
4. We need to implement Transaction Validation Mechanism to ensure the all the transactions are valid before putting them into a block

## Reference
https://hackernoon.com/learn-blockchains-by-building-one-117428612f46  
https://www.youtube.com/watch?v=b81Ib_oYbFk